<a href="https://colab.research.google.com/github/sera0911/asia_ai_study/blob/main/New_MachinLearning/prj_01_news_category_classfication_02_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 네이버 기사 헤드라인을 크롤링한 데이터 전처리

In [ ]:
! pip install konlpy

In [ ]:
#모듈 불러오기

import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt

In [ ]:
pd.set_option('display.unicode.east_asian_width', True)  #줄맞춤을 보기에 간결하게 해준다

In [ ]:
df = pd.read_csv('/content/datasets/naver_news_titles_210616.csv', index_col=0) 
print(df.head())
print(df.info())

                                                                title  category
0    김총리  새 거리두기 방안  일상에 큰 변화 될 것                    Politics
1     백신 유급휴가비  지원한다 법개정안 복지위 통과                   Politics
2   쇄신   변화  강조한 송영길  개혁 속도조절  국민공감대 우선 ......  Politics
3    소통수석  열린자세로  의 중요 이웃국 역할할 것  종합 ......       Politics
4             큐어백  백신 도입     한국  생산 거점                    Politics
<class 'pandas.core.frame.DataFrame'>
Int64Index: 28309 entries, 0 to 28308
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     28309 non-null  object
 1   category  28309 non-null  object
dtypes: object(2)
memory usage: 663.5+ KB
None


In [ ]:
#중복된 기사가 있을 수 있어서 중복된 데이터들은 삭제해준다

col_dup = df['title'].duplicated()  #duplicated=중복이라면 true
print(col_dup)
sum_dup = df.title.duplicated().sum()  #중복 개수 확인
print(sum_dup)

0        False
1        False
2        False
3        False
4        False
         ...  
28304    False
28305    False
28306    False
28307    False
28308    False
Name: title, Length: 28309, dtype: bool
1234


In [ ]:
df = df.drop_duplicates(subset=['title'])  #타이틀이 중복이라면 중복된 row 제거
sum_dup = df.title.duplicated().sum()  #제거 후 다시 중복 개수 확인
print(sum_dup)

0


In [ ]:
#중복이 제거되면 데이터프레임 인덱스가 빠지게 되어 다시 정렬

df.reset_index(drop=True, inplace=True)  #원본을 바꿔주기 위해 둘 다 true로 준다

In [ ]:
#데이터를 나눠준다(피쳐, 타켓값으로)
X = df['title']
Y = df['category']

#### Y 전처리

In [ ]:
#Y(타겟)는 문자열로 들어가있기에 라벨로 바꿔준다

encoder = LabelEncoder()
labeled_Y = encoder.fit_transform(Y)  #데이터를 라벨로 바꿔준다 (문자열 6가지를 같은 문자끼리 라벨로 묶어준다)
label = encoder.classes_  #인코더에 등록된 클래스 조회
print(label)  #카테고리 이름확인

['Culture' 'Economic' 'IT' 'Politics' 'Social' 'World']


In [ ]:
#라벨 순서를 기억해야 해서 따로 저장해 준다
#원래 숫자를 파일로 저장하면 문자열로 저장되지만 ,pickle을 사용하면 데이터 형태 그대로 저장된다(문자 or 숫자 or plt등등)
import pickle 

with open('/content/datasets/category_encoder.pickle', 'wb') as f:  #f란 이름으로 열어준다
    pickle.dump(encoder, f)  #dump= 토큰을 f에 저장시켜준다

In [ ]:
print(labeled_Y)  #라벨화 시킨 카테고리가 잘변환됐는지 확인

[3 3 3 ... 2 2 2]


In [ ]:
#라벨이 붙은 걸 원핫인코딩으로 변환 

onehot_Y = to_categorical(labeled_Y)  #희소행렬화
print(onehot_Y)

[[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 ...
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


#### X 전처리

In [ ]:
#자연어인 X를 전처리해준다(한글 형태소 분류기 사용 - Okt)

okt = Okt()
print(type(X))
print(X[0])  #원본 기사 제목
okt_X = okt.morphs(X[0])  #첫번째 기사 제목만 넣어보기, okt.morphs= 텍스트를 형태소 단위로 나누는데, 이때 각 단어에서 어간을 추출
print(okt_X) 
okt_X = okt.pos(X[0])  #각 단어의 품사를 태깅해준다
print(okt_X)  #okt화 시킨 것(리스트로 들어가진다)

<class 'pandas.core.series.Series'>
총리 거리 두기 방안 일상 변화
['총리', '거리', '두기', '방안', '일상', '변화']
[('총리', 'Noun'), ('거리', 'Noun'), ('두기', 'Noun'), ('방안', 'Noun'), ('일상', 'Noun'), ('변화', 'Noun')]


1. okt = Okt() - 객체 생성  
2. okt.morphs() - 텍스트를 형태소 단위로 나눈다. 옵션으로는 norm과 stem이 있다
- norm은 normalize의 약자로 문장을 정규화하는 역할
- stem은 각 단어에서 어간을 추출하는 기능
3. okt.nouns() - 텍스트에서 명사만 뽑아낸다.
4. okt.phrases() - 텍스트에서 어절을 뽑아낸다.
5. okt.pos() - 각 품사를 태깅하는 역할.
- 품사를 태깅한다는 것은 주어진 텍스트를 형태소 단위로 나누고, 나눠진 각 형태소를 그에 해당하는 품사와 함

In [ ]:
# for문을 사용해서 X전체 텍스트를 형태소 단위로 분리시키기 

for i in range(len(X)):
    X[i] = okt.morphs(X[i])
print(X)

0        [김, 총리, 새, 거리, 두기, 방안, 일상, 에, 큰, 변화, 될, 것]...
1        [백신, 유급, 휴가, 비, 지원, 한, 다, 법, 개정안, 복지, 위, 통과]...
2        [쇄신, 변화, 강조, 한, 송영길, 개혁, 속도, 조절, 국민, 공감, 대, 우선...
3        [소통, 수석, 열린, 자세, 로, 의, 중요, 이웃, 국, 역할, 할, 것, 종합...
4                 [큐어, 백, 백신, 도입, 한국, 생산, 거점]
                               ...                        
27070    [2만, 4000년, 간, 죽지, 않은, 좀비, 가, 시베리아, 에서, 나타났다]...
27071    [PLAY, IT, 갤럭시, 북, 경험, 키, 보드, 에서도, 블루투스, 키, 보드...
27072    [PLAY, IT, 카카오, 판, 클럽, 하우스, 음, 체험, 기, 작, 지만, 큰...
27073    [PLAY, IT, 홈쇼핑, 고객, 정착, 가능할까, CJ, 온스타일, 이용, 해보...
27074    [PLAY, IT, 크롬, 잡겠다는, 네이버, 웨일, 신, 기능, 사용, 해보니]...
Name: title, Length: 27075, dtype: object


In [ ]:
# 형태소로 잘라 낸 뒤, 의미 없는 조사, 접속사, 감탄사 같은 단어들은 제거하기 위해 이 단어들을 모아놓은 파일 가져오기

stopwords = pd.read_csv('/content/datasets/stopwords.csv')
print(stopwords.head(30))

    Unnamed: 0  stopword
0            0        아
1            1        휴
2            2    아이구
3            3    아이쿠
4            4    아이고
5            5        어
6            6        나
7            7      우리
8            8      저희
9            9      따라
10          10      의해
11          11        을
12          12        를
13          13        에
14          14        의
15          15        가
16          16      으로
17          17        로
18          18      에게
19          19    뿐이다
20          20  의거하여
21          21  근거하여
22          22  입각하여
23          23  기준으로
24          24    예하면
25          25      예를
26          26      들면
27          27    들자면
28          28        저
29          29      소인


In [ ]:
#stopwords로 불필요한 단어 제거하기

words = []  #빈리스트 생성
for word in okt_X:
    if word not in list(stopwords['stopword']):  #단어가 stopwords안에 있지 않다면 words 리스트에 추가하기
        words.append(word)

print(words)

[('김', 'Noun'), ('총리', 'Noun'), ('새', 'Noun'), ('거리', 'Noun'), ('두기', 'Noun'), ('방안', 'Noun'), ('일상', 'Noun'), ('에', 'Josa'), ('큰', 'Verb'), ('변화', 'Noun'), ('될', 'Verb'), ('것', 'Noun')]


In [ ]:
#한 글자인 단어와, stopwords로 불필요한 단어 제거하기

for i in range(len(X)):  #i는 각각의 문장들
    result = []
    for j in range(len(X[i])):  #j는 각 문장 안 단어들
        if len(X[i][j]) > 1:  #한글자인 단어들은 if문 안에 못들어간다
            if X[i][j] not in list(stopwords['stopword']):  #그리고 stopwords에 없는 단어들만 들어간다
                result.append(X[i][j])  #통과된 텍스트를 빈 리스트에 넣어주기
    X[i] = ' '.join(result)  #문장 별 리스트 요소들을 join으로 이어준다
    
print(X)

0                            총리 거리 두기 방안 일상 변화
1                     백신 유급 휴가 지원 개정안 복지 통과
2           쇄신 변화 강조 송영길 개혁 속도 조절 국민 공감
3                  소통 수석 열린 자세 중요 이웃 역할 종합
4                            큐어 백신 도입 한국 생산 거점
                               ...                        
27070          2만 4000년 죽지 않은 좀비 시베리아 나타났다
27071    PLAY IT 갤럭시 경험 보드 에서도 블루투스 보드 트리오 500 보니...
27072                 PLAY IT 카카오 클럽 하우스 체험 차이
27073    PLAY IT 홈쇼핑 고객 정착 가능할까 CJ 온스타일 이용 해보니...
27074    PLAY IT 크롬 잡겠다는 네이버 웨일 기능 사용 해보니...
Name: title, Length: 27075, dtype: object


In [ ]:
#토크나이징 작업하기(숫자로 라벨화 작업)

token = Tokenizer()
token.fit_on_texts(X)  #토크나이저에서는 fit_transform이 아닌 fit_on_texts를 사용하여 변환한다, 각 단어들에게 라벨을 붙여준다(token 번호만 가지고 있는다)
tokened_X = token.texts_to_sequences(X)  #texts_to_sequences = 단어를 시퀀스형태로 변환하기(문장변화는 여기서 일어난다)

print(tokened_X[0])  #첫줄만 확인하기

[281, 74, 138, 873, 603, 126]


In [ ]:
#토큰 저장하기(학습한 문장을 라벨을 붙였기에 그대로 저장해준다)

with open('/content/datasets/news_token.pickle', 'wb') as f:  #f란 이름으로 열어준다
    pickle.dump(token, f)  #dump= 토큰을 f에 저장시켜준다

In [ ]:
wordsize = len(token.word_index) + 1   #word_index = 각 단어 별 붙여진 번호를 볼 수 있다, len을 써서 붙인 단어의 개수를 출력함(0이 없어서 0을 추가하기 위해서 1을 더해준다)
print(wordsize)

24151


In [ ]:
#문장별로 단어의 갯수가 달라서 맞춰주기

max = 0
for i in range(len(tokened_X)):
    if max < len(tokened_X[i]):  #문장을 하나하나 꺼내서 길이를 보면서 가장 큰 문장을 찾아준다
        max = len(tokened_X[i])  #가장 긴 문장을 다시 max값으로 지정

print(max)  #최대 문장 길이= 27

27


In [ ]:
#16개의 길이를 맞추기 위해 짧은 문장은 앞쪽에 0으로 채워준다

X_pad = pad_sequences(tokened_X, max)  #pad_sequences= max가 안되는 문장을 0으로 채워준다(앞쪽에)
print(X_pad[:10])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0   281    74   138
    873   603   126]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     2  4851  1001   105
   1953  2506   189]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0  2507   126  1030    66   636   219
   2825     7  1245]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0   923  1803  1804  3229  1421
   1547  1095     4]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0  1805     2   312
     11   156  2661]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0    66    19  1139 13780  3725  1139
   1059

#### 훈련, 결과 데이터 나눠서 저장

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_pad, onehot_Y, test_size=0.1)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(24367, 27)
(2708, 27)
(24367, 6)
(2708, 6)


In [ ]:
#나눈 데이터 저장하기

xy = X_train, X_test, Y_train, Y_test
np.save('/content/datasets/news_data_max_{}_size_{}'.format(max, wordsize), xy)  #max, wordsize는 전체 뉴스기사를 넣었을 땐 달라질 수 있기에 {}안에 넣어준다

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
